In [2]:
!pip install opendatasets --quiet

In [3]:
import pandas as pd
import numpy as np
import opendatasets as od
import email

In [4]:
url = (
    "https://www.kaggle.com/datasets/wcukierski/enron-email-dataset"
)

od.download(url)

100%|██████████| 358M/358M [00:04<00:00, 79.1MB/s]


In [5]:
!ls -lh ./enron-email-dataset/

total 1.4G
-rw-r--r-- 1 root root 1.4G Jan  7 09:54 emails.csv


In [6]:
emails = pd.read_csv("./enron-email-dataset/emails.csv")
emails.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [7]:
emails.shape

(517401, 2)

In [9]:
print(emails['message'][0])

Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 
